In [1]:
# 1. Install necessary libraries
!pip install langchain langchain-openai langchain-core -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 2.3 MB/s eta 0:00:00


WARNING: Package(s) not found: langchain-community
Name: langchain-core
Version: 0.3.79
Summary: Building applications with LLMs through composability
Home-page:
Author:
Author-email:
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: jsonpatch, langsmith, packaging, pydantic, PyYAML, tenacity, typing-extensions
Required-by: langchain, langchain-openai, langchain-text-splitters
---
Name: langchain
Version: 0.3.27
Summary: Building applications with LLMs through composability
Home-page:
Author:
Author-email:
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by:

In [2]:
!pip show langchain-community langchain-core langchain

Name: langchain-core
Version: 0.3.79
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: jsonpatch, langsmith, packaging, pydantic, PyYAML, tenacity, typing-extensions
Required-by: langchain, langchain-openai, langchain-text-splitters
---
Name: langchain
Version: 0.3.27
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: 


In [3]:
import langchain
print(langchain.__version__)

0.3.27


In [4]:
import os
# Read the key from file
with open("openai_key.txt", "r") as f:
    api_key = f.read().strip()

# Set it as an environment variable
os.environ["OPENAI_API_KEY"] = api_key

In [17]:
import os
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.tools import tool # Modern decorator for tools

In [26]:
# instantiating the base llm object

#from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI

myllmObject = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    api_key=os.environ["OPENAI_API_KEY"]
)

In [27]:
#creating country-capital tool

In [28]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("What is the capital of {country}?")

parser = StrOutputParser()

countrycapitalchain = prompt | myllmObject | parser

response = countrycapitalchain.invoke({"country": "Japan"})
print(response)



The capital of Japan is Tokyo.


In [42]:
from langchain.tools import tool
# Assuming countrycapitalchain.run is your function to get capital of a country
@tool
def countrycapital_tool(query: str) -> str:
    """Useful for when you need to answer questions about a country and its capital."""
    #return countrycapitalchain.run(query)
    return countrycapitalchain.invoke({"country":query})

In [30]:
# When giving tools to an LLM, pass as a list
tools = [countrycapital_tool]

In [31]:
tools[0].name, tools[0].description

('countrycapital_tool',
 'Useful for when you need to answer questions about a country and its capital.')

In [ ]:
# Creating a small Math tool -- start

In [32]:
prompt = ChatPromptTemplate.from_template("Solve this equation {equation}?")

parser = StrOutputParser()

mathchain = prompt | myllmObject | parser

response = mathchain.invoke({"equation": "3x=9"})
print(response)

To solve the equation 3x = 9, we need to isolate the variable x.

Divide both sides of the equation by 3:

3x/3 = 9/3
x = 3

Therefore, the solution to the equation 3x = 9 is x = 3.


In [55]:
from langchain.tools import tool
# Assuming countrycapitalchain.run is your function to get capital of a country
@tool
def maths_tool(query: str) -> str:
    """Useful for when you need to answer qsimple equations."""
    return mathchain.invoke({"equation": query})


In [34]:
# When giving tools to an LLM, pass as a list
tools.append(maths_tool)

In [ ]:
# End of Creating Math tool -- End

In [35]:
print(tools[0].name, tools[0].description)
print(tools[1].name, tools[1].description)


countrycapital_tool Useful for when you need to answer questions about a country and its capital.
maths_tool Useful for when you need to answer qsimple equations.


In [ ]:
#del tools[0]

In [36]:
tools

[StructuredTool(name='countrycapital_tool', description='Useful for when you need to answer questions about a country and its capital.', args_schema=<class 'langchain_core.utils.pydantic.countrycapital_tool'>, func=<function countrycapital_tool at 0x7995ff2c98a0>),
 StructuredTool(name='maths_tool', description='Useful for when you need to answer qsimple equations.', args_schema=<class 'langchain_core.utils.pydantic.maths_tool'>, func=<function maths_tool at 0x7995ff2ca340>)]

In [37]:
# --- 2. Initialize the LLM ---

# Use a model that supports Tool Calling (e.g., gpt-4o-mini, gpt-4)
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
myllmObject = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    api_key=os.environ["OPENAI_API_KEY"]
)

In [49]:
# This structure is critical for passing history and scratchpad
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an extremely helpful assistant. Use the 'maths_tool' tool only when asked about the linear equations and  countrycapital_tool  tool for country capital"),
        # This is where the Agent Executor injects past messages (memory)
        MessagesPlaceholder(variable_name="chat_history"),
        # This is the new user input
        ("human", "{input}"),
        # This is where the Agent Executor injects the LLM's thoughts and tool observations
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [50]:
# --- 4. Assemble the Agent (LCEL / Pipeline) ---

# A. Create the Agent: The runnable that decides what action to take (LLM + Tools + Prompt)
agent = create_tool_calling_agent(myllmObject, tools, prompt)

# B. Create the Executor: The runtime that manages the loop (Agent + Execution logic)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=False # See the reliable tool-calling JSON flow
)

In [51]:
# --- 5. Define Conversation History (Memory) ---
chat_history = [
    HumanMessage(content="Hello! My name is John."),
    AIMessage(content="Hello John! How can I help you today?"),
]

In [52]:
# --- 6. Run the Agent ---

print("\n--- Turn 1: Tool Use ---")
result_1 = agent_executor.invoke({
    "input": "Solve the linear 3x+9=6",
    "chat_history": chat_history # Pass the history explicitly
})
print(f"\n🤖 AI Response 1: {result_1['output']}")


--- Turn 1: Tool Use ---

🤖 AI Response 1: The solution to the linear equation 3x + 9 = 6 is x = -1.


In [53]:
chat_history.extend([
    HumanMessage(content="What is the capital of united Kingdom?"),
    AIMessage(content=result_1['output'])
])

In [54]:
print("\n--- Turn 2: Tool Use ---")
result_2 = agent_executor.invoke({
    "input": "What is the capital of United Kingdom",
    "chat_history": chat_history # Pass the updated history
})
print(f"\n🤖 AI Response 2: {result_2['output']}")


--- Turn 2: Tool Use ---

🤖 AI Response 2: The capital of the United Kingdom is London.


In [56]:
print("\n--- Turn 3: Memory Check (No Tool Use) ---")
result_3= agent_executor.invoke({
    "input": "What was the name I told you?",
    "chat_history": chat_history # Pass the updated history
})
print(f"\n🤖 AI Response 2: {result_3['output']}")


--- Turn 3: Memory Check (No Tool Use) ---

🤖 AI Response 2: You told me your name is John.
